In [63]:
import pandas as pd
import numpy as np
import datarobot as dr
import requests
import sys
import os
from pprint import pprint

API_TOKEN = os.getenv('DATAROBOT_API_TOKEN')
ENDPOINT = os.getenv('DATAROBOT_ENDPOINT')
USERNAME = 'matthew.cohen@datarobot.com'

In [152]:
#
# Check the health status of a deploymen and replace the model if failing
#
DEPLOYMENT_ID = '5bb7a726b11ba4576ba28c1f'  # James Johnston's baseball
DEPLOYMENT_ID = '5bdf672f7c6f8b2939428077'  # My recommended XGBoost @ 80%

headers2 = {'Content-Type': 'application/json', 'Authorization': 'token %s' % API_TOKEN}
health_response = requests.get('%s/modelDeployments/%s/' % (ENDPOINT, DEPLOYMENT_ID), headers=headers2)
print(health_response)

dr.Client(endpoint=ENDPOINT, token=API_TOKEN)

# This will error as "Project doesn't exist if you don't have permission for the deployment model's project
if health_response.json()['modelHealth'] == 'failing':
    # Get the model
    model = dr.Model.get(model_id=health_response.json()['model']['id'], 
                         project=health_response.json()['project']['id'])
    # Create a new project on new data
    retrainProject = dr.Project.start(sourcedata='data/10K_Lending_Club_Loans.csv', 
                                      project_name='Lending Club Retrain', target='is_bad',
                                      autopilot_on=False)  # if you want the same model, or...
                                      # autopilot_on=True)  # if you want to use the new best model.  More code required
    # Retrain model on the same blueprint
    modelJobId = retrainProject.train(model.blueprint)
    newModel = dr.models.modeljob.wait_for_async_model_creation(project_id=retrainProject.id, 
                                                                model_job_id=modelJobId)
    # Run feature impact
    fi = newModel.get_or_request_feature_impact(600)
    
    # The patch sends the new model id to be associated with the deployment endpoint?
    model_Update = requests.patch('%s/v2/modelDeployments/%s/model' % (ENDPOINT, DEPLOYMENT_ID), headers=headers2, 
                                  data="{'modelId':'%s'}" % newModel.id)
    pprint.pprint(model_Update)

# pprint(health_response.json())

<Response [200]>


In [18]:
model = dr.Model.get(model_id=health_response.json()['model']['id'], 
                            project=health_response.json()['project']['id'])

In [19]:
model

Model('eXtreme Gradient Boosted Trees Classifier with Early Stopping (Fast Feature Binning)')

In [38]:
#
# Score new data on the deployment
#
# Read in scoring file.  In this case, we'll read the source, sample a few rows, and drop the target
path = '/Users/matthew.cohen/Dev/DataRobot-my github repo/Baseball/'
df = pd.read_csv(path+'pitch_scoring.csv', parse_dates=['date'])
df_pred = df.sample(10, random_state=1)
df_pred.drop(['strike'], axis=1, inplace=True)

# **Writing back out to file a read back in to get the request.post to work.  
# Not ideal, tests below have failed thus far.
pred_file = path+'pitch_pred.csv'
df2.to_csv(pred_file)

data = open(pred_file, 'rb').read()  # This works.  This is type bytes: print(type(data))
print(data)

# Make predictions on your data
# The URL has the following format:
#     https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/<DEPLOYMENT_ID>/predictions
# See docs for details:
#     app.datarobot.com/docs/users-guide/deploy/api/new-prediction-api.html
predictions_response = requests.post('https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/%s/predictions' % (DEPLOYMENT_ID),
                                     auth=(USERNAME, API_TOKEN), data=data, headers=headers)

predictions_response.raise_for_status()
print(predictions_response.json())

In [39]:
df2 = df.sample(10, random_state=1)
df2.drop(['strike'], axis=1, inplace=True)
df2.to_csv(path+'pitch_pred.csv')

In [ ]:
df2.to_csv()

<class 'bytes'>
<class 'bytes'>


## Dev

In [ ]:
df2.to_json()
print(type(data))
print(type(df2.to_msgpack()))
df2.to_msgpack()

In [ ]:
rec = df2.to_records(index=False)

print(repr(rec))
# rec.array([(10, 18446744073709551615, 13240000000.0), (15, 230498234019, 3.14159),
#  (20, 32094812309, 234.1341)], 
#           dtype=[('a', '|u1'), ('b', '<u8'), ('c', '<f8')])

s = rec.tostring()
rec2 = np.fromstring(s, rec.dtype)

print(np.all(rec2 == rec))

In [ ]:
# j = df2.to_json()
# j.encode()
# # pprint(j.encode())

# data = json.loads(j)
# s = json.dumps(data, indent=4, sort_keys=True)
# print(s)

df2.to_string().encode()

In [ ]:
type(data2)
zxc = open(path+'pitch_pred.csv', 'rb')
type(zxc.read())

In [148]:
# Model deployment integrations example
# Set HTTP headers
# Note: The charset should match the contents of the file.
headers = {'Content-Type': 'text/plain; charset=UTF-8', 'datarobot-key': '544ec55f-61bf-f6ee-0caf-15c7f919a45d'}

# data = open(sys.argv[1], 'rb').read()

# data = df2.to_msgpack()  # Fails as bad request

# df2.to_csv(b'asf')   # Fails as can't make a bytes object

# import io
# buffer = io.StringIO()
# df2.to_csv(buffer)
# data = buffer.read()  # Fails bd request
# data = open(, 'rb').read()

# headers = {'Content-Type': 'application/json', 'datarobot-key': '544ec55f-61bf-f6ee-0caf-15c7f919a45d'}
# data= df2.to_json()  # Fails bad request

import json
# data = json.dumps(df2.to_json())
j = df2.to_json()

import re
data2 = df2.to_string()
# data = re.sub("\s+", ",", data.strip())  # remove new lines, 422
# data = re.sub("\s+[-[\r\n]]", ",", data.strip())  # remove new lines, 422
# data = re.sub("\s-[\r\n]", ",", data.strip())  # remove new lines, 422
# data = re.sub("[^\S\x0a\x0d]", ",", data.strip())
data2 = ',' + re.sub(" +", ",", data2.strip())

# # data = data.replace(' ', ',')
data2 = data2.encode('utf-8')  # fails, bad request

data = open(path+'pitch_pred.csv', 'rb').read()  # This works.  This is type bytes: print(type(data))
print(data)

# Make predictions on your data
# The URL has the following format:
#     https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/<DEPLOYMENT_ID>/predictions
# See docs for details:
#     app.datarobot.com/docs/users-guide/deploy/api/new-prediction-api.html
predictions_response = requests.post('https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/%s/predictions' % (DEPLOYMENT_ID),
                                     auth=(USERNAME, API_TOKEN), data=data, headers=headers)

predictions_response.raise_for_status()
print(predictions_response.json())

b',date,ax,ay,az,batter_side,break_angle,break_length,break_y,end_speed,nasty,pfx_x,pfx_z,pitch_type,pitch_type_seq,pitcher_hand,px,pz,spin_dir,spin_rate,start_speed,sz_bot,sz_top,type_conf,vx0,vy0,vz0,weather,wind,x0,y0,z0,zone\n608088,2017-07-19,-11.763477623139696,21.840163814967397,-32.001962831328,R,17.4,9.5,23.8,75.3,44.0,-8.186862024628452,0.11976408107841552,CH,|CH,R,1.29943363056269,2.20153945359716,269.16200000000003,1435.991,82.0,1.5639499340547405,3.446745544178897,2.0,9.731902755506301,-118.85919120808799,-1.2898657962525,"90 degrees, clear","16 mph, Out to RF",-1.7759893884264402,50.0,5.645431611656691,14.0\n675695,2017-08-20,-18.8528107218,28.9955898727,-21.2320240018,R,39.5,6.4,23.8,82.7,58.0,-10.635139710599999,6.172552282650001,FT,|FT,R,-0.8384398279620001,2.70260383171,239.86900000000003,2387.012,91.2,1.7259849379031422,3.524394221724993,2.0,4.66854348858,-132.484583495,-1.9674981673900005,"73 degrees, roof closed","0 mph, None",-1.24440565766,50.0,5.01077974435,11.0